In [1]:
import pandas as pd

In [2]:
df_thread = pd.read_csv('thread_18cff0b946f3a4d4_cleaned.csv', index_col='Unnamed: 0')
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 76 to 100
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   threadId          11 non-null     object
 1   from              11 non-null     object
 2   date              11 non-null     object
 3   body_cleaned      11 non-null     object
 4   body_cleaned_len  11 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [3]:
df_thread = df_thread.reset_index().drop('index', axis=1)
df_thread

,threadId,from,date,body_cleaned,body_cleaned_len
0,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 15:59:44-03:00,"Prezada DGBD, boa tarde!\r\n\r\nEstamos com er...",105
1,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 16:29:16-03:00,Copiando para o e-mail correto da DIBD.\r\n\r\...,15
2,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 17:09:03-03:00,Parece ser de tnsnames.ora desatualizado na es...,36
3,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:29:30-03:00,"Obrigado, Edson.",2
4,18cff0b946f3a4d4,Evandro Bassanesi <evandro.bassanesi@trt4.jus.br>,2024-01-12 17:34:50-03:00,"Felipe,\r\n\r\nEste está desatualizado. O banc...",24
5,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:46:45-03:00,Vamos atualizar. Obrigado!\r\n\r\n\r\nAtencios...,18
6,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 08:58:54-03:00,"Felipe, isso não ocorre no servidor do Markvis...",104
7,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 09:19:39-03:00,"Agora que entendi, o problema que refere, não...",46
8,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 10:18:12-03:00,A GPO de atualização do TNSnames (desktop-all-...,28
9,18cff0b946f3a4d4,Francisco Flesch <francisco.flesch@trt4.jus.br>,2024-01-15 11:05:28-03:00,Eu troquei o TNSnames na última vez que deu pr...,41


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [7]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n\n",  # Split character (default \n\n)
    chunk_size=100,
    chunk_overlap=1,
    length_function=len,
)

In [9]:
text = """
Prezada DGBD, boa tarde!

Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem
dar uma olhada, por favor?


[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""
> argument(s): ""ORA-12541: TNS: nÆo h  listener""




Atenciosamente,

Felipe Giacomel
Divisão de Microinformática
TRT4 - Secretaria de Tecnologia da Informação e Comunicações

-- 
You received this message because you are subscribed to the Google Groups ""SETIC-DIM- Divisao de Microinformatica"" group.
To unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.
"""

In [13]:
splits = splitter.split_text(text)

Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 208, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100


In [21]:
strings = { k:v for (k,v) in zip(range(len(splits)), splits) }
strings

{0: 'Prezada DGBD, boa tarde!',
 1: 'Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem\ndar uma olhada, por favor?',
 2: '[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦\n> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""\n> argument(s): ""ORA-12541: TNS: nÆo h  listener""',
 3: 'Atenciosamente,',
 4: 'Felipe Giacomel\nDivisão de Microinformática\nTRT4 - Secretaria de Tecnologia da Informação e Comunicações',
 5: '-- \nYou received this message because you are subscribed to the Google Groups ""SETIC-DIM- Divisao de Microinformatica"" group.\nTo unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.'}

In [19]:
prompt_grader_importante_system = """
    You are a helpful assistant who is in charge of classifying strings as either interesting to a use case or not.
    This strings will be used by another llm to generate a report and a knowledge base fo a IT company.
    So the use case is to determine if the string contain helpful information to create a knowledge base or not.

    You should output a json. Key shold be the key of the string, the value should be the boolean value
"""


prompt_grader_importante_user = """
    {strings}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_grader = ChatPromptTemplate.from_messages(
    [("system", prompt_grader_importante_system), ("user", prompt_grader_importante_user)]
)

In [20]:
chain_grader = prompt_grader | llm

In [22]:
response = chain_grader.invoke({"strings": strings})
response

AIMessage(content='{\n    "0": false,\n    "1": true,\n    "2": true,\n    "3": false,\n    "4": false,\n    "5": false\n}', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 361, 'total_tokens': 399}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a100994a-49a0-4e9a-bda9-5f77128ff69b-0', usage_metadata={'input_tokens': 361, 'output_tokens': 38, 'total_tokens': 399})

In [24]:
print( response.content )

{
    "0": false,
    "1": true,
    "2": true,
    "3": false,
    "4": false,
    "5": false
}


In [26]:
strings

{0: 'Prezada DGBD, boa tarde!',
 1: 'Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem\ndar uma olhada, por favor?',
 2: '[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦\n> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""\n> argument(s): ""ORA-12541: TNS: nÆo h  listener""',
 3: 'Atenciosamente,',
 4: 'Felipe Giacomel\nDivisão de Microinformática\nTRT4 - Secretaria de Tecnologia da Informação e Comunicações',
 5: '-- \nYou received this message because you are subscribed to the Google Groups ""SETIC-DIM- Divisao de Microinformatica"" group.\nTo unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.'}

In [27]:
prompt_grader_importante_system = """
    You are a helpful assistant who is in charge of classifying strings as either interesting to a use case or not.
    This strings will be used by another llm to generate a report and a knowledge base fo a IT company.
    So the use case is to determine if the string contain helpful information to create a knowledge base or not.

    You should output a json. Key shold be the key of the string, the value should be a tuple with the boolean value and the explaination of why it is important.
"""


prompt_grader_importante_user = """
    {strings}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_grader = ChatPromptTemplate.from_messages(
    [("system", prompt_grader_importante_system), ("user", prompt_grader_importante_user)]
)

In [28]:
chain_grader = prompt_grader | llm

In [29]:
response = chain_grader.invoke({"strings": strings})
response

AIMessage(content='{\n    "0": [false, "This string is a simple greeting and does not contain any relevant information for a knowledge base."],\n    "1": [true, "This string contains information about an error while performing operations on the Infor database in São Leopoldo, which can be helpful for troubleshooting and creating a knowledge base."],\n    "2": [true, "This string provides detailed information about a connection failure to the Infor database in São Leopoldo, including error messages and user information, which is important for diagnosing and resolving the issue."],\n    "3": [false, "This string is a simple closing remark and does not contain any relevant information for a knowledge base."],\n    "4": [false, "This string contains information about the sender\'s name, division, and organization, but it is not directly related to creating a knowledge base."],\n    "5": [false, "This string is a standard email footer and does not contain any relevant information for a know

In [30]:
print( response.content )

{
    "0": [false, "This string is a simple greeting and does not contain any relevant information for a knowledge base."],
    "1": [true, "This string contains information about an error while performing operations on the Infor database in São Leopoldo, which can be helpful for troubleshooting and creating a knowledge base."],
    "2": [true, "This string provides detailed information about a connection failure to the Infor database in São Leopoldo, including error messages and user information, which is important for diagnosing and resolving the issue."],
    "3": [false, "This string is a simple closing remark and does not contain any relevant information for a knowledge base."],
    "4": [false, "This string contains information about the sender's name, division, and organization, but it is not directly related to creating a knowledge base."],
    "5": [false, "This string is a standard email footer and does not contain any relevant information for a knowledge base."]
}


In [31]:
strings

{0: 'Prezada DGBD, boa tarde!',
 1: 'Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem\ndar uma olhada, por favor?',
 2: '[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦\n> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""\n> argument(s): ""ORA-12541: TNS: nÆo h  listener""',
 3: 'Atenciosamente,',
 4: 'Felipe Giacomel\nDivisão de Microinformática\nTRT4 - Secretaria de Tecnologia da Informação e Comunicações',
 5: '-- \nYou received this message because you are subscribed to the Google Groups ""SETIC-DIM- Divisao de Microinformatica"" group.\nTo unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.'}

In [ ]:
prompt_report_creator_system = """
    You are an expert copy-writer that write email conversation reports.
    You will receive two inputs: a knowledge base document of a IT company and a list of strings extracted from a conversation between IT Technicians.
    You should consolidate the strings into the knowledge base document.
    Some strings should by copied char by char, like log strings, error messages.
    The language of the conversation is generaly in Brazilian Portuguese. So your summary should be in Brazilian Portuguese.
    Dont write uninformative phrases like "the email discusses".
    """

prompt_report_creator_user = """
    PREVIOUS REPORT BELOW
    {REPORT}
    
    STRINGS TO BE CONSOLIDATED BELOW
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_report = ChatPromptTemplate.from_messages(
    [("system", prompt_report_creator_system), ("user", prompt_report_creator_user)]
)

In [ ]:
chain_report = prompt_report | llm